# Pseudo-Labeling Flow

This notebook implements an automated pseudo-labeling pipeline designed to streamline the annotation process for object detection and instance segmentation tasks. The tool iteratively improves model performance by using an initial model trained on a small set of manually annotated data to generate labels on new images, which can then be refined and used to retrain progressively better models.

**A "flow" represents a complete pseudo labeling run with specific configuration settings (model type, initial dataset size, correction strategy), while "iterations" are the individual training cycles within each flow where new data is added and the model is retrained.**

The features within this notebook include:
- **Automated Pipeline**: Complete workflow from data preparation to model training
- **Database Logging**: Database tracking for all iterations
- **CVAT Integration**: For viewing and adjusting annotations
- **Flexible Configuration**: Supports different model architectures and training settings
- **Status Monitoring**: Real-time pipeline status and progress tracking



Automatic Export to CVAT only tested and functional for Instance Segmentation + Object Detection

## Imports

In [ ]:
import onedl.client

from pseudo_labeling import PseudoLabelingPipeline

## Global Initalizers
Configure the pipeline with your project-specific settings:


In [ ]:
pipeline = PseudoLabelingPipeline(
    project_name="daniel-osman---streamlining-annotation-bootstrapping/pipeline-test",
    main_dataset_name="full-dataset:0", #input only
    initial_annotated_dataset_name="initial-annotations:0",
    validation_dataset="val:0",
    sample_size_per_iter=150,
    current_flow = 0,
    min_confidence=0.5,
    local_path='',
    cvat_project_id=88,#Ignore for now
    db_path="pseudo_labeling_metadata.db"
)

print("Pipeline initialized")


## Training Config
Set up your model training parameters interactively:


In [ ]:
# Option 1: Interactive widget setup (uncomment to use)
# train_cfg = pipeline.setup_training_config()


# Option 2: Direct dictionary configuration (recommended for specific config)
pipeline.train_cfg = {
    'model_type': 'FasterRCNNConfig',
    'task_type': 'object_detection',
    'backbone': 'RESNET_50',
    'epochs': 1,
    'batch_size': 3,
}

print("Training configuration set:")
print(pipeline.train_cfg)


# (1) Initial Flow, Training, and Evaluation Setup

This step initiates the current flow and establishes the baseline model using your initial annotated dataset.
1. Loads the initial annotated dataset.
2. Created training set for the current flow.
3. Trains the first baseline model (iteration 0) for the specified flow.
4. Evaluates Model
5. Logs Metadata (Only variables generated throughout the process of the pipeline are 'predicted_dataset_name', 'model_uid', 'evaluation_uid', 'evaluation_info')

⚠️ ATTENTION: Skip this section if your current flow already exists and if you already have a baseline model

In [ ]:
pipeline.get_pipeline_status()

### 1.1 Train Initial Model and Evaluate on Validation Set
Train the baseline model on your initial annotated dataset.
Evaluate the initial model performance on the validation dataset.




If you want to use an existing model, then run: <span style="color:#d73a49; font-family:monospace;">pipeline.log_iteration_0_external_model("smoky-shepherd-0")</span>, then skip to section (2):


In [ ]:
pipeline.train_model()
pipeline.evaluate_model()

### 1.2 Log
Save all metadata for the initial training iteration to the database.


In [ ]:
pipeline.log_iteration_0() # This is required now but will be removed
print("Initial model training and evaluation complete, Current status:")

# (2) Pseudo-Labeling Iteration Workflow

This step executes a complete pseudo-labeling iteration cycle using the model from the previous iteration to generate labels on new data.
1. Sets up the next iteration with correction strategy (manual or automated).
2. Samples new unlabeled data from the full dataset.
3. Runs inference using the previous iteration's model to generate pseudo-labels.
4. Handles corrections based on strategy: exports to CVAT for manual corrections OR merges pseudo-labels directly.
5. Trains updated model on expanded dataset (original + new data).
6. Evaluates the updated model performance.
7. Logs iteration metadata to track progress and results.

**⚠️ ATTENTION: Set 'manual_corrections=True' for CVAT workflow with human review, or 'manual_corrections=False' for fully automated pseudo-labeling**


In [ ]:
pipeline.get_pipeline_status()

### Local Initializer
Configure the next iteration parameters

Set `manual_corrections = True` for CVAT manual review.

Set `manual_corrections = False` for fully automated pseudo-labeling.


In [ ]:
manual_corrections = False
pipeline.setup_next_iteration(manual_corrections)


### 2.1 Sample New Data

In [ ]:
pipeline.sample_unseen_inputs()


### 2.2 Generate Predictions/Pseudo-Labels


In [ ]:
pipeline.run_inference()

### 2.3 CVAT Export
Run even if manual correction is false.

In [ ]:
if pipeline.manual_corrections_global:
    print("Manual corrections enabled - proceeding to CVAT export")
    pipeline.manually_correct_cvat()
    print("After completing corrections in CVAT, manually update the predicted dataset and run the merge cell below")
else:
    print("No Manual Correction, Proceed to merging the datasets")


### 2.4. Merge Data
This cell merges the dataset with current training set. If **manual_correction = True**, then corrected annotations will be exported and merged.


In [ ]:
pipeline.merge_pseudo_labels()

### 2.5 Train Updated Model
Train a new model on the expanded training set

In [ ]:
pipeline.train_model()

### 2.6 Evaluate Performance

In [ ]:
pipeline.evaluate_model()

### 2.7 Status

In [ ]:
pipeline.get_pipeline_status()

# Additional Runs
To run additional iterations, repeat Section 2 after logging. For creating a new flow, go back to Section 1, update the current_flow and go again.
